In [1]:
import time
from os import path
from pathlib import Path
from datetime import datetime
import pandas as pd

import astropy.units as u
from astropy.table import Table, vstack, unique

from astroquery.ipac.ned import Ned
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
#from astroquery.ned.core import RemoteServiceError, TableParseError

import requests
from requests import Request, Session

from numpy import unique as uniq

from astropy.coordinates import SkyCoord
from astropy.table import Table

In [2]:
path_raw='../Datos/Raw/'
path_phot_data = '../Datos/Phot_Tables_NED/' #In this path was saved the photometry tables for each object
Sample = Table.read(path_raw+'SMB_Mar_31_2022_Sy_Samples.vot',format='votable')
Sample

main_id,ra,dec,coo_bibcode,otype_txt,rvz_bibcode,rvz_radvel,rvz_redshift,rvz_type,nbref,otypes,cds_url
,deg,deg,,,,km / s,,,,,
object,float64,float64,object,object,object,float64,float64,str1,int32,object,str93
RX J1203.5+4432,180.88724599190996,2.4929432724600002,2020yCat.1350....0G,Sy1,2012ApJS..203...21A,22264.155457144552,0.07724,z,42,* |AGN|EmO|G |GiC|IR |QSO|Rad|Sy1|V* |X,http://vizier.u-strasbg.fr/viz-bin/sed?-c=180.88724599190996%202.4929432724600002&-c.rs=5
2MASX J12014361+4307386,180.43169828928998,43.1273394439,2020yCat.1350....0G,Sy2,2009ApJS..182..543A,20748.633978793205,0.07178,z,4,* |G |Rad|Sy2|rG,http://vizier.u-strasbg.fr/viz-bin/sed?-c=180.43169828928998%2043.1273394439&-c.rs=5
SDSS J144452.42+511957.5,221.218431,51.332651,2012ApJS..203...21A,Sy2,2015ApJS..219...12A,25457.504881087,0.08885,z,4,G |GiG|Sy2,http://vizier.u-strasbg.fr/viz-bin/sed?-c=221.218431%2051.332651&-c.rs=5
FIRST J083344.2+283939,128.43421094374,28.66111602879,2020yCat.1350....0G,Sy1,2009ApJS..182..543A,173513.43668282355,0.936,z,13,* |AGN|QSO|Rad|Sy1,http://vizier.u-strasbg.fr/viz-bin/sed?-c=128.43421094374%2028.66111602879&-c.rs=5
SDSS J091658.99+012604.8,139.24580903083,1.43466726489,2020yCat.1350....0G,Sy1,2015ApJS..219...12A,125231.50632150602,0.56039,z,8,* |QSO|Sy1,http://vizier.u-strasbg.fr/viz-bin/sed?-c=139.24580903083%201.43466726489&-c.rs=5
MCG+06-34-028,235.23225416666668,32.78155555555555,2006AJ....131.1163S,Sy2,2002LEDA.........0P,15618.951821140114,0.05353,z,12,G |IR |Sy2,http://vizier.u-strasbg.fr/viz-bin/sed?-c=235.23225416666668%2032.78155555555555&-c.rs=5
IC 3834,192.88488958362,-14.221116888659996,2020yCat.1350....0G,Sy1,2002LEDA.........0P,4332.318180333962,0.014557,z,34,* |AGN|G |GiG|IR |Sy1,http://vizier.u-strasbg.fr/viz-bin/sed?-c=192.88488958362%20-14.221116888659996&-c.rs=5
SDSS J155011.46+233017.1,237.54777578073,23.50475679549,2020yCat.1350....0G,Sy2,2012ApJS..203...21A,33510.40645237621,0.11879,z,1,* |Sy2,http://vizier.u-strasbg.fr/viz-bin/sed?-c=237.54777578073%2023.50475679549&-c.rs=5


In [39]:
cat_smb_sample = SkyCoord(ra=Sample['ra'], dec=Sample['dec'])

In [3]:
Sample[30037]

main_id,ra,dec,coo_bibcode,otype_txt,rvz_bibcode,rvz_radvel,rvz_redshift,rvz_type,nbref,otypes,cds_url
,deg,deg,,,,km / s,,,,,
object,float64,float64,object,object,object,float64,float64,str1,int32,object,str93
2C 349,59.72682085409999,10.43410884888,2020yCat.1350....0G,Sy2,2011MNRAS.416.2840L,9130.0,0.0309325926265982,v,330,AGN|G |Rad|Sy2|X,http://vizier.u-strasbg.fr/viz-bin/sed?-c=59.72682085409999%2010.43410884888&-c.rs=5


In [11]:
other_ids = Simbad.query_objectids(Sample['main_id'][43670])
other_ids

ID
bytes26
IRAS 18368+3549
LEDA 90303
2MASX J18383543+3552197
PSCz Q18368+3549
[VV2010c] J183835.4+355220
[DG2000] 18368+3549
IRAS Z18368+3549


In [5]:
i=30037
phot_ned = Table.read(path_phot_data+Sample['main_id'][i]+'_NED_phot_tables'+'.vot', 
                      format='votable')
phot_ned

No.,Observed_Passband,Photometry_Measurement,Uncertainty,Units,Frequency,Flux_Density,Upper_limit_of_uncertainty,Lower_limit_of_uncertainty,Upper_limit_of_Flux_Density,Lower_limit_of_Flux_Density,NED_Uncertainty,NED_Units,Refcode,Significance,Published_frequency,Frequency_Mode,Coordinates_Targeted,Spatial_Mode,Qualifiers,Comments
,,,,,Hz,Jy,,,,,,,,,,,,,,
int32,object,float64,object,object,float64,float64,float64,float64,float64,float64,object,object,object,object,object,object,object,object,object,object
1,15-150 keV (Swift),6.8e-12,+/-4.1e-12,erg/cm^2^/s,1.99e+19,3.42e-08,2.06e-08,2.06e-08,--,--,+/-2.06E-08,Jy,2010A&A...524A..64C,uncertainty,82.50 keV,Broad-band measurement,059.678 +10.402 (J2000),Modelled datum,,From new raw data; NED frequency assigned to mid-point ofband in keV
2,2-10 keV (XMM),2.62e-12,,erg/cm^2^/s,1.45e+18,1.81e-07,--,--,--,--,,Jy,2009ApJ...705..454N,no uncertainty reported,6.00 keV,Broad-band measurement,,Total flux,,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
3,2-7 keV (Chandra),6.8298e-13,+/-20.18E-15,erg/cm^2^/s,1.09e+18,6.27e-08,1.85e-09,1.85e-09,--,--,+/-1.85E-09,Jy,2015ApJS..220....5M,1 sigma,4.50 keV,Broad-band measurement,,Flux integrated from map,,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
4,0.3-8 keV (Chandra),7.79e-13,,erg/cm^2^/s,1e+18,7.79e-08,--,--,--,--,,Jy,2016ApJS..224...40W,no uncertainty reported,4.15 keV,Broad-band measurement,,Flux integrated from map,Average flux,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
5,2-7 keV Chandra,6.7412e-15,+/-5.6626E-15,erg/cm^2/s,9.19e+17,7.34e-10,6.16e-10,6.16e-10,--,--,+/-6.16E-10,Jy,2020CSC...C2..0000:,68% confidence,3.8 keV,Broad-band measurement,,Flux in fixed aperture,flux_aper;ext,From new raw data
6,2-7 keV Chandra,6.7412e-15,+/-5.6626E-15,erg/cm^2/s,9.19e+17,7.34e-10,6.16e-10,6.16e-10,--,--,+/-6.16E-10,Jy,2020CSC...C2..0000:,68% confidence,3.8 keV,Broad-band measurement,,Flux in fixed aperture,flux_aper_avg;ext,From new raw data
7,2-7 keV Chandra,1.9973e-15,+/-2.1083E-15,erg/cm^2/s,9.19e+17,2.17e-10,2.29e-10,2.29e-10,--,--,+/-2.29E-10,Jy,2020CSC...C2..0000:,68% confidence,3.8 keV,Broad-band measurement,,From fitting to map,flux_aper90;ext,From new raw data
8,2-7 keV Chandra,1.9973e-15,+/-2.1083E-15,erg/cm^2/s,9.19e+17,2.17e-10,2.29e-10,2.29e-10,--,--,+/-2.29E-10,Jy,2020CSC...C2..0000:,68% confidence,3.8 keV,Broad-band measurement,,From fitting to map,flux_aper90_avg;ext,From new raw data


In [7]:
def photNED(name):
    
    phot_ned = Table()
    id_used=[]
    
    try:
        phot_ned = Ned.get_table(name, table='photometry') #We get the Photometry Table using first SIMBAD names
        id_used = name
    except:
        other_ids = Simbad.query_objectids(name) #We get other IDs for the object from SIMBAD
        for other_id in other_ids['ID']:
            try:
                phot_ned = Ned.get_table(other_id, table='photometry') #Photometry Table using other SIMBAD names
                id_used = other_id
                break
            except:
                if other_id == other_ids['ID'][-1]:
                    continue
    return phot_ned, id_used

In [8]:
Sample['main_id'][i]

'2C  349'

In [9]:
phot_t,name_ned = photNED(Sample['main_id'][30037])
phot_t

No.,Observed Passband,Photometry Measurement,Uncertainty,Units,Frequency,Flux Density,Upper limit of uncertainty,Lower limit of uncertainty,Upper limit of Flux Density,Lower limit of Flux Density,NED Uncertainty,NED Units,Refcode,Significance,Published frequency,Frequency Mode,Coordinates Targeted,Spatial Mode,Qualifiers,Comments
,,,,,Hz,Jy,,,,,,,,,,,,,,
int32,object,float64,object,object,float64,float64,float64,float64,float64,float64,object,object,object,object,object,object,object,object,object,object
1,15-150 keV (Swift),6.8e-12,+/-4.1e-12,erg/cm^2^/s,1.99e+19,3.42e-08,2.06e-08,2.06e-08,--,--,+/-2.06E-08,Jy,2010A&A...524A..64C,uncertainty,82.50 keV,Broad-band measurement,059.678 +10.402 (J2000),Modelled datum,,From new raw data; NED frequency assigned to mid-point ofband in keV
2,2-10 keV (XMM),2.62e-12,,erg/cm^2^/s,1.45e+18,1.81e-07,--,--,--,--,,Jy,2009ApJ...705..454N,no uncertainty reported,6.00 keV,Broad-band measurement,,Total flux,,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
3,2-7 keV (Chandra),6.8298e-13,+/-20.18E-15,erg/cm^2^/s,1.09e+18,6.27e-08,1.85e-09,1.85e-09,--,--,+/-1.85E-09,Jy,2015ApJS..220....5M,1 sigma,4.50 keV,Broad-band measurement,,Flux integrated from map,,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
4,0.3-8 keV (Chandra),7.79e-13,,erg/cm^2^/s,1e+18,7.79e-08,--,--,--,--,,Jy,2016ApJS..224...40W,no uncertainty reported,4.15 keV,Broad-band measurement,,Flux integrated from map,Average flux,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
5,2-7 keV Chandra,6.7412e-15,+/-5.6626E-15,erg/cm^2/s,9.19e+17,7.34e-10,6.16e-10,6.16e-10,--,--,+/-6.16E-10,Jy,2020CSC...C2..0000:,68% confidence,3.8 keV,Broad-band measurement,,Flux in fixed aperture,flux_aper;ext,From new raw data
6,2-7 keV Chandra,6.7412e-15,+/-5.6626E-15,erg/cm^2/s,9.19e+17,7.34e-10,6.16e-10,6.16e-10,--,--,+/-6.16E-10,Jy,2020CSC...C2..0000:,68% confidence,3.8 keV,Broad-band measurement,,Flux in fixed aperture,flux_aper_avg;ext,From new raw data
7,2-7 keV Chandra,1.9973e-15,+/-2.1083E-15,erg/cm^2/s,9.19e+17,2.17e-10,2.29e-10,2.29e-10,--,--,+/-2.29E-10,Jy,2020CSC...C2..0000:,68% confidence,3.8 keV,Broad-band measurement,,From fitting to map,flux_aper90;ext,From new raw data
8,2-7 keV Chandra,1.9973e-15,+/-2.1083E-15,erg/cm^2/s,9.19e+17,2.17e-10,2.29e-10,2.29e-10,--,--,+/-2.29E-10,Jy,2020CSC...C2..0000:,68% confidence,3.8 keV,Broad-band measurement,,From fitting to map,flux_aper90_avg;ext,From new raw data


In [10]:
name_ned

'1Jy 0356+10'

In [64]:
result_t = Ned.query_region(cat_smb_sample[505],radius=5 * u.arcsec)
id_ned = result_t["Object Name"] 
id_ned

LQAC 196+028 008


In [65]:
phot_ta = Ned.get_table(id_ned, table='photometry')

RemoteServiceError: The remote service returned the following error message.
ERROR:  No Photometric data points found.

In [12]:
req = requests.get(Sample['cds_url'][283], 
                   headers={'cache-control': 'private, max-age=0, no-cache', "Pragma": "no-cache"})
if req.status_code == 200:
    with open(Sample['main_id'][283]+'.vot', 'wb') as f_vot:
        f_vot.write(req.content)
        f_vot.close()

In [22]:
cds_table = Table.read(Sample['main_id'][283]+'.vot', format='votable')

In [29]:
cds_table[40:60]

_RAJ2000,_DEJ2000,_tabname,_ID,_tab1_31,_tab1_32,sed_freq,sed_flux,sed_eflux,sed_filter
deg,deg,,,d,s,GHz,Jy,Jy,
float64,float64,str32,str64,float64,float64,float64,float32,float32,str32
150.457106620,28.786526357,I/350/gaiaedr3,"-c=150.45710661986 +28.78652635685,eq=ICRS&-c.rs=0.004",--,0.0,393400,5.87e-05,9.6e-06,GAIA/GAIA3:Grp
150.457106620,28.786526357,I/350/gaiaedr3,"-c=150.45710661986 +28.78652635685,eq=ICRS&-c.rs=0.004",--,0.0,514900,1.06e-05,2e-07,GAIA/GAIA3:G
150.457106620,28.786526357,I/350/gaiaedr3,"-c=150.45710661986 +28.78652635685,eq=ICRS&-c.rs=0.004",--,0.0,595300,1.7e-05,3.6e-06,GAIA/GAIA3:Gbp
150.456344196,28.785809265,I/353/gsc242,"-c=150.45634514424 +28.78581193911,eq=ICRS&-c.rs=0.004",--,0.0,13571,0.299,0.005,WISE:W4
150.456344196,28.785809265,I/353/gsc242,"-c=150.45634514424 +28.78581193911,eq=ICRS&-c.rs=0.004",--,0.0,25934,0.112,0.002,WISE:W3
150.456344196,28.785809265,I/353/gsc242,"-c=150.45634514424 +28.78581193911,eq=ICRS&-c.rs=0.004",--,0.0,65172,0.0248,0.0005,WISE:W2
150.456344196,28.785809265,I/353/gsc242,"-c=150.45634514424 +28.78581193911,eq=ICRS&-c.rs=0.004",--,0.0,89490,0.0114,0.0002,WISE:W1
150.456344196,28.785809265,I/353/gsc242,"-c=150.45634514424 +28.78581193911,eq=ICRS&-c.rs=0.004",--,0.0,138550,0.005,0.00015,2MASS:Ks


In [25]:
t=Table(names=['a',], dtype=['O',])

In [26]:
t

a
object
